In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import time

In [2]:
plt.style.use("ggplot")
plt.rcParams["figure.figsize"] = (12, 8)

In [3]:
%run cards.ipynb
deck = Deck()
deck

<Deck: 312 cards>

In [8]:
%run game.ipynb

In [9]:
%run visualization.ipynb
d = Visualization()

Output()